In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
lobby = pd.read_csv('../csv/lobbied_bills_no_duplicates.csv')
bills = pd.read_csv('../csv/cleaned_bills.csv')

In [13]:
bills.id

0         119.S.2403
1         119.S.1728
2         119.S.1440
3         119.S.2283
4        119.HR.5371
            ...     
60134      113.S.159
60135      113.S.155
60136      113.S.285
60137       113.S.28
60138      113.S.255
Name: id, Length: 60139, dtype: object

In [9]:
lobby.b_id

0        hr5376-117
1         hr748-116
2        hr3684-117
3        hr1319-117
4        hr6800-116
            ...    
72983     ZZ5531694
72984     ZZ2687988
72985     ZZ2681752
72986     ZZ2653910
72987     ZZ5541460
Name: b_id, Length: 72988, dtype: object

In [19]:
import re
def transform_bid(b_id):
    """
    Transforms a b_id string from format like 'hr6270-116' or 'sjres8-116'
    to '116.HR.6270' or '116.SJRES.8'.
    
    Returns np.nan if the format is not recognized.
    """
    if not isinstance(b_id, str):
        return np.nan

    # Split the string at the hyphen to separate bill part and congress
    parts = b_id.split('-')
    if len(parts) != 2:
        # Return NaN if the format isn't 'something-congress'
        return np.nan  

    bill_part, congress = parts
    
    # Use regex to find the split point between letters and numbers
    # ^([a-zA-Z]+) = Capture group 1: one or more letters at the start
    # (\d+)$       = Capture group 2: one or more digits at the end
    match = re.match(r'^([a-zA-Z]+)(\d+)$', bill_part)
    if not match:
        # Return NaN if the bill_part format isn't 'lettersNUMBERS'
        return np.nan  

    chamber_prefix = match.group(1).upper() # e.g., 'hr' -> 'HR'
    bill_number = match.group(2)          # e.g., '6270'

    # Reconstruct the new ID in the target format
    new_id = f"{congress}.{chamber_prefix}.{bill_number}"
    return new_id

# Apply the transformation to create the new 'id' column in the lobby DataFrame
lobby['id'] = lobby['b_id'].apply(transform_bid)


lobby['id'] = lobby['id'].astype(str)
bills['id'] = bills['id'].astype(str)

# Perform the merge
# Using 'inner' merge to keep only rows where the ID exists in both dataframes
merged_df = pd.merge(lobby, bills, on='id', how='inner')

In [25]:
merged_df[['total_amount', 'total_lobbyists', 'total_agencies','passed_senate', 'passed_house', 'to_president', 'law']].corr()

,total_amount,total_lobbyists,total_agencies,passed_senate,passed_house,to_president,law
total_amount,1.000000,0.948172,0.918706,0.087724,0.087724,0.087724,0.087724
total_lobbyists,0.948172,1.000000,0.986101,0.102380,0.102380,0.102380,0.102380
total_agencies,0.918706,0.986101,1.000000,0.104884,0.104884,0.104884,0.104884
passed_senate,0.087724,0.102380,0.104884,1.000000,1.000000,1.000000,1.000000
passed_house,0.087724,0.102380,0.104884,1.000000,1.000000,1.000000,1.000000
to_president,0.087724,0.102380,0.104884,1.000000,1.000000,1.000000,1.000000
law,0.087724,0.102380,0.104884,1.000000,1.000000,1.000000,1.000000


In [32]:
merged_df.describe()

c:\Anaconda\envs\WillABill\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,congno,total_amount,total_lobbyists,total_agencies,first_year,last_year,num_years,Unnamed: 0,congress,number,house_democrat_members,house_independent_members,house_republican_members,senate_democrat_members,senate_independent_members,senate_republican_members,avg_amt_per_lobbyist
count,26257.000000,2.625700e+04,26257.000000,26257.000000,26257.000000,26257.000000,26257.000000,26257.000000,26257.000000,26257.000000,26257.000000,26257.000000,26257.000000,26257.000000,26257.000000,26257.000000,2.625500e+04
mean,115.249686,2.021772e+07,164.100773,205.921469,2017.874395,2018.449137,1.572914,41356.107095,115.249686,3266.024222,214.276460,0.503637,233.514339,48.481700,2.173592,52.307309,inf
std,1.473500,8.065908e+07,725.871726,869.484422,2.939193,2.792763,0.608712,11378.083129,1.473500,2296.695299,19.752838,0.868132,17.341659,4.122235,0.563082,3.115938,NaN
min,113.000000,0.000000e+00,0.000000,0.000000,2013.000000,2013.000000,1.000000,8521.000000,113.000000,1.000000,190.000000,0.000000,208.000000,44.000000,2.000000,46.000000,0.000000e+00
25%,114.000000,2.600000e+05,9.000000,16.000000,2015.000000,2016.000000,1.000000,32178.000000,114.000000,1403.000000,200.000000,0.000000,208.000000,46.000000,2.000000,51.000000,2.125013e+04
50%,115.000000,1.400000e+06,30.000000,50.000000,2018.000000,2018.000000,2.000000,41714.000000,115.000000,2882.000000,204.000000,0.000000,240.000000,48.000000,2.000000,54.000000,5.000000e+04
75%,116.000000,8.390000e+06,101.000000,153.000000,2020.000000,2020.000000,2.000000,51212.000000,116.000000,4768.000000,241.000000,2.000000,250.000000,49.000000,2.000000,54.000000,1.119271e+05
max,118.000000,4.881270e+09,60679.000000,67856.000000,2023.000000,2023.000000,5.000000,60138.000000,118.000000,9709.000000,241.000000,2.000000,251.000000,57.000000,4.000000,55.000000,inf


In [36]:
merged_df['avg_amt_per_lobbyist'] = merged_df.total_amount / merged_df.total_lobbyists
merged_df[(merged_df.total_amount > 0)&(merged_df.total_lobbyists > 0)].avg_amt_per_lobbyist.describe()

count    2.563900e+04
mean     8.384219e+04
std      1.168645e+05
min      3.768293e+02
25%      2.273369e+04
50%      5.200000e+04
75%      1.138746e+05
max      3.456667e+06
Name: avg_amt_per_lobbyist, dtype: float64

In [39]:
total_2024 = merged_df[merged_df.first_year == 2013].total_amount.sum()

In [41]:
total_2024 # > $60B which contradicts open secrets own website!!!!

np.float64(60952808552.0)

Sara's lobbying data is definitely wrong. Will have to fix...